In [2]:
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from ipyvtklink.viewer import ViewInteractiveWidget

In [3]:
thrx_path = "/home/mahsadibaji/mdsc689-03-notebooks/Thorax"
head_gauss_path = "/home/mahsadibaji/mdsc689-03-notebooks/headGaussian.nii"
head_saltpepper_path = "/home/mahsadibaji/mdsc689-03-notebooks/headSaltPepper.nii"

In [23]:
class vtk_pipeline:
    
    def __init__(self, path, filetype,window, level, zslice, render_size = 300):
        self.path = path
        self.filetype = filetype
        self.window = window
        self.level = level
        self.render_size = render_size
        self.slice = zslice
        
        self.out = None
        self.dim = None
        self.img_np = None
        self.image_reader()
        
        self.gauss_filtered_img = None
        self.median_filtered_img = None
        
        self.resizers = {}
        self.mappers = {}
        self.actors = {}
        
        self.renderer = None
        self.render_window = None
        self.interactor = None
        
    def image_reader(self):   
        if self.filetype.lower() == 'dicom':  # Load a DICOM image from file into a vtk pipeline.
            img_reader = vtk.vtkDICOMImageReader()
            img_reader.SetDirectoryName(self.path)
            img_reader.Update()
            self.out = img_reader.GetOutput()
            self.dim = self.out.GetDimensions()
            
        elif self.filetype.lower() == 'niftii': # Load a NIFTI image from file into a vtk pipeline.
            img_reader = vtk.vtkNIFTIImageReader()
            img_reader.SetFileName(self.path)
            img_reader.Update()
            self.out = img_reader.GetOutput()
            self.dim = self.out.GetDimensions()
            
        self.img_np = vtk_to_numpy(self.out.GetPointData().GetScalars()).reshape(self.dim,order='F')
            
    def gaussian_filtering(self):
        
        kernel = np.array([[[1, 2, 1], [2, 4, 2], [1, 2, 1]], [[2, 4, 2], [4, 16, 4], [2, 4, 2]], [[1, 2, 1], [2, 4,
        2], [1, 2, 1]]],dtype=np.float32)
        kernel /= kernel.sum()
        
        padded = np.pad(self.img_np, [(1, 1), (1, 1), (1, 1)], mode='constant', constant_values = 0) #padded by width 1 with zeros

        gauss_filtered_np = np.zeros_like(self.img_np)
        
        for i in range(1,padded.shape[0]-1):
            for j in range(1,padded.shape[1]-1):
                for k in range(1,padded.shape[2]-1):
                        gauss_filtered_np[i-1,j-1,k-1] = np.sum((kernel*padded[i-1:i+2,j-1:j+2,k-1:k+2]))
        
        self.gauss_filtered_img = vtk.vtkImageData()
        self.gauss_filtered_img.DeepCopy(self.out) # here is how to copy a vtkImageData
        self.gauss_filtered_img.GetPointData().SetScalars(numpy_to_vtk(num_array = gauss_filtered_np.ravel(order = 'F'), deep = True))
        
    def median_filtering(self):
        
        median_filtered_np = np.zeros_like(self.img_np)
        
        padded = np.pad(self.img_np, [(1, 1), (1, 1), (1, 1)], mode='constant')

        for i in range(1,padded.shape[0]-1):
            for j in range(1,padded.shape[1]-1):
                for k in range(1,padded.shape[2]-1):
                    median_filtered_np[i-1,j-1,k-1] = np.median(padded[i-1:i+2,j-1:j+2,k-1:k+2].ravel())

        self.median_filtered_img = vtk.vtkImageData()
        self.median_filtered_img.DeepCopy(self.out) # here is how to copy a vtkImageData
        self.median_filtered_img.GetPointData().SetScalars(numpy_to_vtk(num_array = median_filtered_np.ravel(order = 'F'), deep = True))
        
    def create_mousewheel_callbacks(self,mapper):
        
        def move_forward(obj=None,event=None):
            mapper.SetZSlice(mapper.GetZSlice()+1)

        def move_backward(obj=None,event=None):
            mapper.SetZSlice(mapper.GetZSlice()-1)

        return move_forward, move_backward
            
    def create_resizer_mapper_actor(self,data_output):

        resizer = vtk.vtkImageResize()
        resizer.SetInputData(data_output)
        resizer.Update()

        original_dims = np.asarray(resizer.GetOutput().GetDimensions())
        new_dims = list( (original_dims*(self.render_size/original_dims[0:1].max())).astype(int) )

        resizer.SetOutputDimensions(*new_dims)

        mapper = vtk.vtkImageMapper()
        mapper.SetInputData(resizer.GetOutput())
        mapper.SetColorWindow(self.window)
        mapper.SetColorLevel(self.level)
        mapper.SetZSlice(self.slice)

        actor = vtk.vtkActor2D()
        actor.SetMapper(mapper)

        return resizer, mapper, actor
            
    def render(self,filter_type = None):
        
        self.resizers["image"], self.mappers["image"], self.actors['image'] = self.create_resizer_mapper_actor(self.out)
        print("image actor created")

        if filter_type.lower() == "gaussian":
            self.gaussian_filtering()
            print("gaussian filter applied")
            self.resizers["filtered"], self.mappers['filtered'], self.actors['filtered'] = self.create_resizer_mapper_actor(self.gauss_filtered_img)
            print("filter actor created")
        elif filter_type.lower() == "median":
            self.median_filtering()
            print("median filter applied")
            self.resizers["filtered"], self.mappers['filtered'], self.actors['filtered'] = self.create_resizer_mapper_actor(self.median_filtered_img)
            print("filter actor created")
    
        
        self.render_window = vtk.vtkRenderWindow()
        self.render_window.SetOffScreenRendering(True)
        self.render_window.SetSize(self.render_size*2, self.render_size)

        self.renderer = vtk.vtkRenderer()
        self.renderer.ResetCamera()        
        self.render_window.AddRenderer(self.renderer)

        self.interactor = vtk.vtkRenderWindowInteractor()
        self.interactor.SetRenderWindow(self.render_window)
        
        offset = 0
        for actor in self.actors.values():
            self.renderer.AddActor(actor)
            actor.GetPositionCoordinate().SetValue(offset, 0)
            offset+=self.render_size
            
        
        # add callbacks to the interactor
        for mapper in self.mappers.values():
            fwd, bwd = self.create_mousewheel_callbacks(mapper)
            self.interactor.AddObserver('MouseWheelForwardEvent', fwd)
            self.interactor.AddObserver('MouseWheelBackwardEvent', bwd)
            
        self.render_window.Render()
        self.interactor.Initialize()

# Experiments

## HeadGaussian with Gaussian filter

In [17]:
head_gauss_g = vtk_pipeline(path=head_gauss_path, filetype='niftii', window=1000, level=700, zslice=150, render_size=300)
head_gauss_g.render(filter_type="gaussian")
ViewInteractiveWidget(head_gauss_g.render_window)

image actor created
gaussian filter applied
filter actor created


ViewInteractiveWidget(height=300, layout=Layout(height='auto', width='100%'), width=600)

Gaussian filter applied - window: 1000 - level: 700 - slice: 150

## HeadGaussian with Median filter

In [18]:
head_gauss_m = vtk_pipeline(path=head_gauss_path, filetype='niftii', window=1000, level=700, zslice=150, render_size=300)
head_gauss_m.render(filter_type="median")
ViewInteractiveWidget(head_gauss_m.render_window)

image actor created
median filter applied
filter actor created


ViewInteractiveWidget(height=300, layout=Layout(height='auto', width='100%'), width=600)

Median filter applied - window: 1000 - level: 700 - slice: 150

The HeadGaussian sample dataset has Gaussian noise and the gaussian filter seems to work slightly better. Although the median filter could remove the Gaussian noise reletivly well, however, the Gaussian filter preserves the contrast and details in the image better than the median filter because it performs a distance weighted average for each pixel. The median filter works great in situations where there is obvious outliers in the image which is not the case in this sample dataset. 

## HeadSaltPepper with Gaussian filter

In [19]:
head_saltpepper_g = vtk_pipeline(path=head_saltpepper_path, filetype='niftii',window=700, level=200, zslice=150, render_size=300)
head_saltpepper_g.render(filter_type="gaussian")
ViewInteractiveWidget(head_saltpepper_g.render_window)

image actor created
gaussian filter applied
filter actor created


ViewInteractiveWidget(height=300, layout=Layout(height='auto', width='100%'), width=600)

Gaussian filter applied - window: 700 - level: 200 - slice: 150

## HeadSaltPepper with Median filter

In [20]:
head_saltpepper_m = vtk_pipeline(path=head_saltpepper_path, filetype='niftii',window=700, level=200, zslice=150, render_size=300)
head_saltpepper_m.render(filter_type="median")
ViewInteractiveWidget(head_saltpepper_m.render_window)

image actor created
median filter applied
filter actor created


ViewInteractiveWidget(height=300, layout=Layout(height='auto', width='100%'), width=600)

Median filter applied - window: 700 - level: 200 - slice: 150

On the head dataset with saltpepper noise, the median filter outperforms the gaussian filter significantly in removing the noise and it preserved the image contrast and edges well. The reason is that median filter applies the median of neighbor pixel values to each pixel which can remove the effect of outliers (salt and pepper noise with unusual high intensities in the image). The gaussian filter was not able to remove the saltpepper noise effectively because it takes the average of pixels(weighted by distance) and this is highly impacted by these outliers. 

## Thorax with Gaussian filter

In [21]:
thrx_g = vtk_pipeline(path=thrx_path, filetype='dicom', window=1000, level=40, zslice=150, render_size=500)
thrx_g.render(filter_type='gaussian')
ViewInteractiveWidget(thrx_g.render_window)

image actor created
gaussian filter applied
filter actor created


ViewInteractiveWidget(layout=Layout(height='auto', width='100%'), width=1000)

Gaussian filter applied - window: 1000 - level: 40 - slice: 150

## Thorax with Gaussian filter

In [26]:
thrx_m = vtk_pipeline(path=thrx_path, filetype='dicom', window=1000, level=40, zslice=150, render_size=500)
thrx_m.render(filter_type='median')
ViewInteractiveWidget(thrx_m.render_window)

image actor created
median filter applied
filter actor created


ViewInteractiveWidget(layout=Layout(height='auto', width='100%'), width=1000)

Median filter applied - window: 1000 - level: 40 - slice: 150

For the thorax dataset, the Gaussian filter is working better as it reduces the noise in the image while also maintains the details and edges very well. With the median filter, the image looks more blurry, and it also lost some of the details and contrast of the image.

## User Input
In this part, user can specify the dataset and type of filter they want to apply on it to see the results. window and level are specified for each one manually to get the best result. 

In [25]:
dataset = input("Enter the number of dataset you want to apply the filters on! ( 1.headGaussian 2.headSaltPepper 3.Thorax ) \n")
filter_type = input("enter the filter you want to apply: Gaussian/Median\n").lower()
if dataset == "1":
    dataset_path = head_gauss_path
    file_type = "niftii"
    w=1000
    l=700
    rs = 300
elif dataset == "2":
    dataset_path = head_saltpepper_path
    file_type = "niftii"
    w=700
    l=200
    rs = 300
elif dataset == "3":
    dataset_path = thrx_path
    file_type = "dicom"
    w=1000
    l=40
    rs = 500
else:
    print("Invaild input. please enter one of the available options.")
    
obj = vtk_pipeline(path = dataset_path, filetype=file_type, window= w, level= l, zslice= 150, render_size= rs)
obj.render(filter_type = filter_type)
ViewInteractiveWidget(obj.render_window)

Enter the number of dataset you want to apply the filters on! ( 1.headGaussian 2.headSaltPepper 3.Thorax ) 
2
enter the filter you want to apply: Gaussian/Median
Median
image actor created
median filter applied
filter actor created


ViewInteractiveWidget(height=300, layout=Layout(height='auto', width='100%'), width=600)